In [18]:
import pandas as pd
import warnings
import os
import concurrent.futures

warnings.filterwarnings('ignore')
df = pd.read_csv('data_live_engineer_filtered.csv', encoding='utf-8')


In [19]:
df['result'].value_counts()

result
0    358381
1     61965
Name: count, dtype: int64

In [20]:
df.columns

Index(['minute', 'homeTeam', 'awayTeam', 'goal_home', 'goal_away',
       'shots_home', 'shots_away', 'blockedShots_home', 'blockedShots_away',
       'league', 'corners_home', 'corners_away', 'redcards_home',
       'redcards_away', 'shotsOffgoal_home', 'shotsOffgoal_away',
       'shotsOngoal_home', 'shotsOngoal_away', 'yellowcards_home',
       'yellowcards_away', 'fouls_home', 'fouls_away', 'offsides_home',
       'offsides_away', 'tackles_home', 'tackles_away', 'result', 'match_id',
       'match_result', 'win_rate', 'draw_rate', 'loss_rate', 'homeTeam_win',
       'homeTeam_loss', 'homeTeam_draw', 'win_rate_homeTeam',
       'loss_rate_homeTeam', 'draw_rate_homeTeam', 'awayTeam_win',
       'awayTeam_loss', 'awayTeam_draw', 'win_rate_awayTeam',
       'loss_rate_awayTeam', 'draw_rate_awayTeam', 'f_attack_home',
       'f_attack_away', 'f_defensive_home', 'f_defensive_away', '05ht_home',
       '15ht_home', '25ht_home', '05ht_away', '15ht_away', '25ht_away',
       'possessiontime

In [21]:
import unicodedata

# Define a function to normalize text using unicodedata
def normalize_text_unicode(text):
    text = unicodedata.normalize('NFKD', text)
    text = text.encode('ascii', 'ignore').decode('utf-8')
    return text.lower()

# Apply the function to the relevant columns
df['league'] = df['league'].apply(normalize_text_unicode)
df['homeTeam'] = df['homeTeam'].apply(normalize_text_unicode)
df['awayTeam'] = df['awayTeam'].apply(normalize_text_unicode)
df.head()

,minute,homeTeam,awayTeam,goal_home,goal_away,shots_home,shots_away,blockedShots_home,blockedShots_away,league,...,aggresssionOverTime_away,defensiveEfficacy_home,defensiveEfficacy_away,taklesOverTime_home,taklesOverTime_away,possessionControl,passRisk_home,passRisk_away,TotalCards_home,TotalCards_away
0,1.49,carabobo fc,club universitario de deportes,0.0,0.0,0.0,0.0,0.0,0.0,south america - copa libertadores,...,0.0,0.0,0.0,0.0,0.000000,6.0,0.0,0.0,0.0,0.0
1,1.90,carabobo fc,club universitario de deportes,0.0,0.0,0.0,0.0,0.0,0.0,south america - copa libertadores,...,0.0,0.0,0.0,0.0,0.000000,20.0,0.0,0.0,0.0,0.0
2,2.28,carabobo fc,club universitario de deportes,0.0,0.0,0.0,1.0,0.0,1.0,south america - copa libertadores,...,0.0,0.0,100.0,0.0,0.438596,16.0,0.0,0.0,0.0,0.0
3,3.48,carabobo fc,club universitario de deportes,0.0,0.0,0.0,1.0,0.0,1.0,south america - copa libertadores,...,0.0,0.0,100.0,0.0,0.287356,18.0,0.0,0.0,0.0,0.0
4,3.80,carabobo fc,club universitario de deportes,0.0,0.0,0.0,1.0,0.0,1.0,south america - copa libertadores,...,0.0,0.0,100.0,0.0,0.263158,18.0,0.0,0.0,0.0,0.0


In [22]:
df.columns

Index(['minute', 'homeTeam', 'awayTeam', 'goal_home', 'goal_away',
       'shots_home', 'shots_away', 'blockedShots_home', 'blockedShots_away',
       'league', 'corners_home', 'corners_away', 'redcards_home',
       'redcards_away', 'shotsOffgoal_home', 'shotsOffgoal_away',
       'shotsOngoal_home', 'shotsOngoal_away', 'yellowcards_home',
       'yellowcards_away', 'fouls_home', 'fouls_away', 'offsides_home',
       'offsides_away', 'tackles_home', 'tackles_away', 'result', 'match_id',
       'match_result', 'win_rate', 'draw_rate', 'loss_rate', 'homeTeam_win',
       'homeTeam_loss', 'homeTeam_draw', 'win_rate_homeTeam',
       'loss_rate_homeTeam', 'draw_rate_homeTeam', 'awayTeam_win',
       'awayTeam_loss', 'awayTeam_draw', 'win_rate_awayTeam',
       'loss_rate_awayTeam', 'draw_rate_awayTeam', 'f_attack_home',
       'f_attack_away', 'f_defensive_home', 'f_defensive_away', '05ht_home',
       '15ht_home', '25ht_home', '05ht_away', '15ht_away', '25ht_away',
       'possessiontime

In [26]:
# Arredondando todos os valores decimais para 2 casas decimais
df = df.round(2)

In [28]:
df.shape

(420346, 71)

In [30]:
import numpy as np


# Substitua valores infinitos por NaN e então preencha ou remova NaNs
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)
df.shape

(417662, 71)

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Encode categorical variables
label_encoders = {}
for column in ['league', 'homeTeam', 'awayTeam']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Split the df into features and target variable
X = df.drop(['result', 'match_id'], axis=1)
y = df['result']

# Split the df into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train a Random Forest classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

In [32]:
X.columns

Index(['minute', 'homeTeam', 'awayTeam', 'goal_home', 'goal_away',
       'shots_home', 'shots_away', 'blockedShots_home', 'blockedShots_away',
       'league', 'corners_home', 'corners_away', 'redcards_home',
       'redcards_away', 'shotsOffgoal_home', 'shotsOffgoal_away',
       'shotsOngoal_home', 'shotsOngoal_away', 'yellowcards_home',
       'yellowcards_away', 'fouls_home', 'fouls_away', 'offsides_home',
       'offsides_away', 'tackles_home', 'tackles_away', 'match_result',
       'win_rate', 'draw_rate', 'loss_rate', 'homeTeam_win', 'homeTeam_loss',
       'homeTeam_draw', 'win_rate_homeTeam', 'loss_rate_homeTeam',
       'draw_rate_homeTeam', 'awayTeam_win', 'awayTeam_loss', 'awayTeam_draw',
       'win_rate_awayTeam', 'loss_rate_awayTeam', 'draw_rate_awayTeam',
       'f_attack_home', 'f_attack_away', 'f_defensive_home',
       'f_defensive_away', '05ht_home', '15ht_home', '25ht_home', '05ht_away',
       '15ht_away', '25ht_away', 'possessiontime_away', 'possessiontime_home'

In [28]:
print(report)

              precision    recall  f1-score   support

           0       0.95      1.00      0.97     71757
           1       0.96      0.71      0.82     12313

    accuracy                           0.95     84070
   macro avg       0.96      0.85      0.90     84070
weighted avg       0.95      0.95      0.95     84070



In [33]:
print(report)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     71328
           1       0.99      0.91      0.94     12205

    accuracy                           0.98     83533
   macro avg       0.98      0.95      0.97     83533
weighted avg       0.98      0.98      0.98     83533



In [34]:
import joblib

# Save the label encoders, scaler, and model
joblib.dump(label_encoders, '../models/label_encoders.pkl')
joblib.dump(scaler, '../models/scaler.pkl')
joblib.dump(model, '../models/random_forest_model.pkl')

['../models/random_forest_model.pkl']